In [4]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.utils import shuffle
import tensorflow as tf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from afinn import Afinn
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Input
import tensorflow_hub as hub

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models, optimizers
from itertools import product
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer
from itertools import product
import csv

#Import data
parlvote = pd.read_csv('ParlVote_concat.csv', header=None)
first_row = parlvote.iloc[0].tolist()
parlvote.columns = first_row
parlvote = parlvote.drop(index=0)

#########################BEGIN SAMPLE##################################
parlvote =  parlvote.sample(n=100, random_state=42)
##########################END SAMPLE###################################

rem_token = []
sh_pv_before = parlvote.shape

# Instantiate a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")


# Tokenize the speeches and count tokens
parlvote['token_length'] = parlvote['speech'].apply(lambda x: len(tokenizer.tokenize(x)))

# Filter out rows with more than 512 tokens
max_tokens = 512
parlvote = parlvote[parlvote['token_length'] <= max_tokens]

# Now 'filtered_parlvote' is a DataFrame containing only rows with no more than 512 BERT tokens in the 'speech' column
# Shape after removing neutral
sh_pv_after = parlvote.shape
rem_token.append({"original" : sh_pv_before, ">512 removed" : sh_pv_after})
rem_token_df = pd.DataFrame(rem_token)

# Save the new DataFrame to a CSV file
rem_token_df.to_csv("removed_512.csv", index=False)

# Initialize sentiment analyzers
vader_analyzer = SentimentIntensityAnalyzer()
afinn_analyzer = Afinn()
textblob_analyzer = TextBlob

# Define a function to generate the sentiment scores for each row
def generate_sentiment_scores(row):
    text = row['speech']
    
    # Generate VADER sentiment score
    vader_scores = vader_analyzer.polarity_scores(text)['compound']
    
    # Generate Afinn sentiment score
    afinn_score = afinn_analyzer.score(text)
    
    # Generate TextBlob sentiment score
    textblob_score = textblob_analyzer(text).sentiment.polarity
    
    return pd.Series([vader_scores, textblob_score, afinn_score])


# Apply the function to generate sentiment scores for each row and store the scores in new columns
parlvote[['vader', 'textblob', 'afinn']] = parlvote.apply(generate_sentiment_scores, axis=1)


# Define a function to convert VADER sentiment scores to labels
def vader_score_to_label(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Define a function to convert TextBlob sentiment scores to labels
def textblob_score_to_label(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# Define a function to convert Afinn sentiment scores to labels
def afinn_score_to_label(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'


# Apply the label conversion functions to the sentiment score columns
parlvote['vader'] = parlvote['vader'].apply(vader_score_to_label)
parlvote['textblob'] = parlvote['textblob'].apply(textblob_score_to_label)
parlvote['afinn'] = parlvote['afinn'].apply(afinn_score_to_label)

# Shape before removing neutral
removed = []
before = parlvote.shape

# remove instances where a row contains 'neutral'
pv = parlvote.loc[~((parlvote['vader'] == 'Neutral') | (parlvote['textblob'] == 'Neutral') | (parlvote['afinn'] == 'Neutral'))]
# Create a copy of the DataFrame to avoid modifying the original DataFrame
pv = pv.copy()

# Shape after removing neutral
after = pv.shape
removed.append({"original" : before, "neutral removed" : after})
removed_df = pd.DataFrame(removed)

# Save the new DataFrame to a CSV file
removed_df.to_csv("removed_neutral.csv", index=False)

def label_to_binary(label):
    if label == 'Positive':
        return 1
    else:
        return 0


def str_to_int(label):
    if label == '1':
        return 1
    else:
        return 0


# Convert sentiment labels to binary values for 'vader', 'textblob', and 'afinn' columns
pv['vader'] = pv['vader'].apply(label_to_binary)
pv['textblob'] = pv['textblob'].apply(label_to_binary)
pv['afinn'] = pv['afinn'].apply(label_to_binary)

# Convert the string of 'vote' to integer
pv['vote'] = pv['vote'].apply(str_to_int)


# Select only the relevant columns
selected_columns = ["vote", "vader", "textblob", "afinn"]
df_selected = pv[selected_columns]

# Calculate the percentage of positive and negative scores for each column
percentages = []
for column in df_selected.columns:
    positive = (df_selected[column].sum() / len(df_selected[column])) * 100
    negative = 100 - positive
    percentages.append({"column": column, "positive": positive, "negative": negative})

# Create a new DataFrame with the calculated percentages
percentages_df = pd.DataFrame(percentages)

# Save the new DataFrame to a CSV file
percentages_df.to_csv("percentages.csv", index=False)


# Define a function to one-hot encode a column
def one_hot_encode(df, column):
    df[column + '_positive'] = (df[column] == 1).astype(int)
    df[column + '_negative'] = (df[column] == 0).astype(int)
    return df


# Apply the function to one-hot encode the columns in the 'pv' dataframe
pv = one_hot_encode(pv, 'vote')
pv = one_hot_encode(pv, 'vader')
pv = one_hot_encode(pv, 'textblob')
pv = one_hot_encode(pv, 'afinn')

# # Prepare sliding window and sentence embedding functions
# def sliding_window(text, window_size, stride):
#     num_chunks = max((len(text) - window_size) // stride + 1, 1)
#     chunks = [text[i*stride:i*stride + window_size] for i in range(num_chunks)]
#     return chunks


# def get_sentence_embeddings(embedding_model, data, max_length, window_size, stride, aggregation='mean'):
#     all_embeddings = []
    
#     for text in data:
#         if len(text) > max_length:
#             chunks = sliding_window(text, window_size, stride)
#         else:
#             chunks = [text]

#         # Generate embeddings for each chunk
#         chunk_embeddings = embedding_model.encode(chunks)
        
#         # Aggregate the embeddings
#         if aggregation == 'mean':
#             aggregated_embedding = np.mean(chunk_embeddings, axis=0)
#         elif aggregation == 'max':
#             aggregated_embedding = np.max(chunk_embeddings, axis=0)
#         elif aggregation == 'min':
#             aggregated_embedding = np.min(chunk_embeddings, axis=0)
#         else:
#             raise ValueError(f"Unsupported aggregation method: {aggregation}")

#         all_embeddings.append(aggregated_embedding)

#     return np.array(all_embeddings)

# # Set the parameters for the sliding window
# max_length = 512
# window_size = 400
# stride = 200

# Load a pre-trained BERT model
embedding_model = SentenceTransformer('bert-base-cased')

def print_majority_baselines(df, one_hot_encoded_columns):
    for columns in one_hot_encoded_columns:
        majority_class = df[columns].sum(axis=0).idxmax()
        majority_baseline = df[majority_class].mean()
        
        print(f"Majority baseline for {', '.join(columns)}:")
        print(f"  Majority class: {majority_class}")
        print(f"  Baseline accuracy: {majority_baseline:.2%}\n")

# usage
one_hot_encoded_columns = [
    ['vote_positive', 'vote_negative'],
    ['vader_positive', 'vader_negative'],
    ['textblob_positive', 'textblob_negative'],
    ['afinn_positive', 'afinn_negative']
]

print_majority_baselines(pv, one_hot_encoded_columns)

def majority_baseline(df, columns):
        majority_class = df[columns].sum(axis=0).idxmax()
        majority_baseline = df[majority_class].mean()
        return majority_baseline

vote_majority = majority_baseline(pv, one_hot_encoded_columns[0])
vader_majority = majority_baseline(pv, one_hot_encoded_columns[1])
textblob_majority = majority_baseline(pv, one_hot_encoded_columns[2])
afinn_majority = majority_baseline(pv, one_hot_encoded_columns[3])

# Convert one-hot encoded columns to a NumPy array
vote_labels = pv[['vote_positive', 'vote_negative']].values
vader_labels = pv[['vader_positive', 'vader_negative']].values
textblob_labels = pv[['textblob_positive', 'textblob_negative']].values
afinn_labels = pv[['afinn_positive', 'afinn_negative']].values

# # Generate sentence embeddings
# embeddings = get_sentence_embeddings(embedding_model, pv['speech'], max_length, window_size, stride, aggregation='mean')

# # Create a new DataFrame for the embeddings
# embeddings_df = pd.DataFrame({'embeddings': list(embeddings)})

# # Reset the index of both DataFrames to avoid issues during concatenation
# pv.reset_index(drop=True, inplace=True)
# embeddings_df.reset_index(drop=True, inplace=True)

# # Concatenate the original DataFrame and the embeddings DataFrame
# pv = pd.concat([pv, embeddings_df], axis=1)

# Generate embeddings for the 'speech' column
pv['embeddings'] = parlvote['speech'].apply(lambda x: embedding_model.encode(x, convert_to_numpy=True))

# Now 'parlvote' has a new column 'embeddings' containing the embeddings as numpy arrays


def sentiment_analysis_pipeline(data, train_labels, test_labels, lstm_units, spatial_dropout_rate, dense_units, learning_rate, batch_size, epochs, input_dropout_rate, recurrent_dropout_rate, output_dropout_rate, smote, use_kfold, test_size=0.2, n_splits=5):
    data = np.stack(data, axis=0)  # Convert the lists of embeddings to a NumPy array
    
    if use_kfold:
        # K-fold cross-validation
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        splits = kfold.split(data, np.argmax(train_labels, axis=1))
    else:
        # Train-test split
        train_data, test_data, train_labels, test_labels = train_test_split(data, train_labels, test_size=test_size, stratify=train_labels, random_state=42)
        splits = [(train_data, test_data, train_labels, test_labels)]

    # Store evaluation scores for each fold or train-test split
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for train_index, test_index in splits:
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = train_labels[train_index], test_labels[test_index]

        # Create the SMOTE instance if specified
        if smote:
            smote_sampler = SMOTE(random_state=42)
            X_train, y_train = smote_sampler.fit_resample(X_train, y_train)

        # Define the model
        model = models.Sequential([
            layers.Input(shape=(X_train.shape[1],)),
            layers.Reshape((1, X_train.shape[1])),
            layers.SpatialDropout1D(spatial_dropout_rate),
            layers.Bidirectional(layers.LSTM(lstm_units, dropout=input_dropout_rate, recurrent_dropout=recurrent_dropout_rate)),
            layers.Dense(dense_units, activation='relu'),
            layers.Dropout(output_dropout_rate),
            layers.Dense(y_train.shape[1], activation='softmax')
        ])


        model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        # Predict on test data
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_test_decoded = np.argmax(y_test, axis=-1)

        # Evaluation
        accuracy = accuracy_score(y_test_decoded, y_pred)
        precision = precision_score(y_test_decoded, y_pred, average='weighted')
        recall = recall_score(y_test_decoded, y_pred, average='weighted')
        f1 = f1_score(y_test_decoded, y_pred, average='weighted')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Print average scores
    print(f"Average accuracy: {np.mean(accuracies):.2%}")
    print(f"Average precision: {np.mean(precisions):.2%}")
    print(f"Average recall: {np.mean(recalls):.2%}")
    print(f"Average F1-score: {np.mean(f1_scores):.2%}")
    accuracy = np.mean(accuracies)
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    f1 = np.mean(f1_scores)
    return accuracy, precision, recall, f1


def mlp_baseline_pipeline(data, train_labels, test_labels, smote, use_kfold, test_size=0.2, n_splits=5, dense_units=1, learning_rate=1e-3, batch_size=32, epochs=10):
    data = np.stack(data, axis=0)  # Convert the lists of embeddings to a NumPy array
    if use_kfold:
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        splits = kfold.split(data, np.argmax(train_labels, axis=1))
    else:
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, stratify=labels, random_state=42)
        splits = [(X_train, X_test, y_train, y_test)]

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for split in splits:
        if use_kfold:
            train_index, test_index = split
            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = train_labels[train_index], test_labels[test_index]

        if smote:
            smote_sampler = SMOTE(random_state=42)
            X_train, y_train = smote_sampler.fit_resample(X_train, y_train)

        model = Sequential([
            Dense(dense_units, activation='sigmoid', input_shape=(X_train.shape[1],)),
            Dense(y_train.shape[1], activation='softmax')
        ])

        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_test_decoded = np.argmax(y_test, axis=-1)

        accuracy = accuracy_score(y_test_decoded, y_pred)
        precision = precision_score(y_test_decoded, y_pred, average='weighted')
        recall = recall_score(y_test_decoded, y_pred, average='weighted')
        f1 = f1_score(y_test_decoded, y_pred, average='weighted')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    print(f"Average accuracy: {np.mean(accuracies):.2%}")
    print(f"Average precision: {np.mean(precisions):.2%}")
    print(f"Average recall: {np.mean(recalls):.2%}")
    print(f"Average F1-score: {np.mean(f1_scores):.2%}")
    accuracy = np.mean(accuracies)
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    f1 = np.mean(f1_scores)
    return accuracy, precision, recall, f1
###############################################################################################

# # Hyperparameter optimizations
# search_space = {
#     'lstm_units': [32, 64, 128],
#     'spatial_dropout_rate': [0.1, 0.2, 0.3],
#     'dense_units': [16, 32, 64],
#     'learning_rate': [1e-3, 1e-4],
#     'batch_size': [16, 32],
#     'epochs': [5, 10, 15], 
#     'smote' : [True, False], 
#     'use_kfold' : [True, False]
# }

# # Hyperparameter setup


# def generate_combinations(search_space):
#     keys, values = zip(*search_space.items())
#     return [dict(zip(keys, v)) for v in product(*values)]

# combinations = generate_combinations(search_space)

# # Hyperparamter optimization functie
# def run_pipeline_with_combination(combination, data, train_labels, test_labels):
#     return sentiment_analysis_pipeline(
#         data=data,
#         train_labels=train_labels,
#         test_labels=test_labels,
#         smote=combination['smote'],
#         use_kfold=combination['use_kfold'],
#         lstm_units=combination['lstm_units'],
#         spatial_dropout_rate=combination['spatial_dropout_rate'],
#         dense_units=combination['dense_units'],
#         learning_rate=combination['learning_rate'],
#         batch_size=combination['batch_size'],
#         epochs=combination['epochs']
#     )

# # Define the filename for the CSV file
# filename = 'experiment_results.csv'

# # Define the header for the CSV file
# header = ['Experiment', 'Parameters', 'F1-score']

# # Initialize an empty list to store the results
# results = []

# # Hyperparameter optimization Experiment 1 
# best_combination_exp_1 = None
# best_f1_exp_1 = -1

# for combination in combinations:
#     f1_exp_1 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vote_labels, test_labels=vote_labels)
    
#     if f1_exp_1 > best_f1_exp_1:
#         best_f1_exp_1 = f1_exp_1
#         best_combination_exp_1 = combination

# print(f"Best combination experiment 1: {best_combination_exp_1}")
# print(f"Best F1-score experiment 1: {best_f1_exp_1}")

# # Add the results for Experiment 1 to the results list
# results.append(('Experiment 1', best_combination_exp_1, best_f1_exp_1))

# # Hyperparameter optimization Experiment 2 
# best_combination_exp_2 = None
# best_f1_exp_2 = -1

# for combination in combinations:
#     f1_exp_2 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vader_labels, test_labels=vader_labels)
    
#     if f1_exp_2 > best_f1_exp_2:
#         best_f1_exp_2 = f1_exp_2
#         best_combination_exp_2 = combination

# print(f"Best combination experiment 2: {best_combination_exp_2}")
# print(f"Best F1-score experiment 2: {best_f1_exp_2}")

# # Add the results for Experiment 2 to the results list
# results.append(('Experiment 2', best_combination_exp_2, best_f1_exp_2))

# # Hyperparameter optimization Experiment 3 
# best_combination_exp_3 = None
# best_f1_exp_3 = -1

# for combination in combinations:
#     f1_exp_3 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vader_labels, test_labels=vote_labels)
    
#     if f1_exp_3 > best_f1_exp_3:
#         best_f1_exp_3 = f1_exp_3
#         best_combination_exp_3 = combination

# print(f"Best combination experiment 3: {best_combination_exp_3}")
# print(f"Best F1-score experiment 3: {best_f1_exp_3}")

# # Add the results for Experiment 3 to the results list
# results.append(('Experiment 3', best_combination_exp_3, best_f1_exp_3))

# # Hyperparameter optimization Experiment 4 
# best_combination_exp_4 = None
# best_f1_exp_4 = -1

# for combination in combinations:
#     f1_exp_4 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=textblob_labels, test_labels=textblob_labels)
    
#     if f1_exp_4 > best_f1_exp_4:
#         best_f1_exp_4 = f1_exp_4
#         best_combination_exp_4 = combination

# print(f"Best combination experiment 4: {best_combination_exp_4}")
# print(f"Best F1-score experiment 4: {best_f1_exp_4}")

# # Add the results for Experiment 4 to the results list
# results.append(('Experiment 4', best_combination_exp_4, best_f1_exp_4))

# # Hyperparameter optimization Experiment 5
# best_combination_exp_5 = None
# best_f1_exp_5 = -1

# for combination in combinations:
#     f1_exp_5 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=textblob_labels, test_labels=vote_labels)
    
#     if f1_exp_5 > best_f1_exp_5:
#         best_f1_exp_5 = f1_exp_5
#         best_combination_exp_5 = combination

# print(f"Best combination experiment 5: {best_combination_exp_5}")
# print(f"Best F1-score experiment 5: {best_f1_exp_5}")

# # Add the results for Experiment 5 to the results list
# results.append(('Experiment 5', best_combination_exp_5, best_f1_exp_5))

# # Hyperparameter optimization Experiment 6 
# best_combination_exp_6 = None
# best_f1_exp_6 = -1

# for combination in combinations:
#     f1_exp_6 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=afinn_labels, test_labels=afinn_labels)
    
#     if f1_exp_6 > best_f1_exp_6:
#         best_f1_exp_6 = f1_exp_6
#         best_combination_exp_6 = combination

# print(f"Best combination experiment 6: {best_combination_exp_6}")
# print(f"Best F1-score experiment 6: {best_f1_exp_6}")

# # Add the results for Experiment 6 to the results list
# results.append(('Experiment 6', best_combination_exp_6, best_f1_exp_6))

# # Hyperparameter optimization Experiment 7 
# best_combination_exp_7 = None
# best_f1_exp_7 = -1

# for combination in combinations:
#     f1_exp_7 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=afinn_labels, test_labels=vote_labels)
    
#     if f1_exp_7 > best_f1_exp_7:
#         best_f1_exp_7 = f1_exp_7
#         best_combination_exp_7 = combination

# print(f"Best combination experiment 7: {best_combination_exp_7}")
# print(f"Best F1-score experiment 7: {best_f1_exp_7}")

# # Add the results for Experiment 7 to the results list
# results.append(('Experiment 7', best_combination_exp_7, best_f1_exp_7))

# # Write the results to the CSV file
# with open(filename, 'a', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(header)
#     writer.writerows(results)


###############################################################################################

# Experiments with best parameters
# Define the filename for the CSV file
filename_2 = 'best_experiment_results.csv'

# Define the header for the CSV file
header_2 = ['Experiment', 'Accuracy', 'Precision', 'Recall', 'F1', 'Majority baseline', 'Parameters']

bilstm_parameters_human = "LSTM_units = 64, \n spatial_dropout = 0.3, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=32, \n epochs=10, \n  input_dropout_rate=0.3, \n recurrent_dropout_rate=0.3, \n output_dropout_rate=0.4"
bilstm_parameters_auto = "LSTM_units = 64, \n spatial_dropout = 0.2, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=32, \n epochs=10, \n  input_dropout_rate=0.2, \n recurrent_dropout_rate=0.2, \n output_dropout_rate=0.5"
bilstm_parameters_mix = "LSTM_units = 64, \n spatial_dropout = 0.3, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=32, \n epochs=10, \n  input_dropout_rate=0.3, \n recurrent_dropout_rate=0.3, \n output_dropout_rate=0.5"
mlp_parameters = 'Standard'


# Initialize an empty list to store the results
results_2 = []


## Experiment 1 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vote_labels,
        test_labels=vote_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.0001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.4
)


# Add the results for Experiment 1 - BiLSTM to the results list
results_2.append(('E1 BiLSTM - vote | vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_human))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vote_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.0001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)


# Add the results for Experiment 1 - MLP baseline to the results list
results_2.append(('E1 MLP - vote | vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 2
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vader_labels,
        test_labels=vader_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5
)


# Add the results for Experiment 2 - BiLSTM to the results list
results_2.append(('E2 BiLSTM - vader | vader', accuracy, precision, recall, f1, vader_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vader_labels,
    test_labels=vader_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True,
)


# Add the results for Experiment 2 - MLP baseline to the results list
results_2.append(('E2 MLP - vader | vader', accuracy, precision, recall, f1, vader_majority, mlp_parameters))


## Experiment 3 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vader_labels,
        test_labels=vote_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5
)

# Add the results for Experiment 3 - BiLSTM to the results list
results_2.append(('E3 BiLSTM - vader | vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vader_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)

# Add the results for Experiment 3 - MLP baseline to the results list
results_2.append(('E3 MLP - vader | vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 4 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=textblob_labels,
        test_labels=textblob_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5
)

# Add the results for Experiment 4 - BiLSTM to the results list
results_2.append(('E4 BiLSTM - textblob | textblob', accuracy, precision, recall, f1, textblob_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=textblob_labels,
    test_labels=textblob_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)

# Add the results for Experiment 4 - MLP baseline to the results list
results_2.append(('E4 MLP - textblob | textblob', accuracy, precision, recall, f1, textblob_majority, mlp_parameters))


## Experiment 5 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=textblob_labels,
        test_labels=vote_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5
)

# Add the results for Experiment 5 - BiLSTM to the results list
results_2.append(('E5 BiLSTM - textblob | vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=textblob_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)

# Add the results for Experiment 5 - MLP baseline to the results list
results_2.append(('E5 MLP - textblob | vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 6 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=afinn_labels,
        test_labels=afinn_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5
)

# Add the results for Experiment 6 - BiLSTM to the results list
results_2.append(('E6 BiLSTM - afinn | afinn', accuracy, precision, recall, f1, afinn_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=afinn_labels,
    test_labels=afinn_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)

# Add the results for Experiment 6 - MLP baseline to the results list
results_2.append(('E6 MLP - afinn | afinn', accuracy, precision, recall, f1, afinn_majority, mlp_parameters))

## Experiment 7 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=afinn_labels,
        test_labels=vote_labels,
        smote=True,
        use_kfold=True,
        lstm_units=64,
        spatial_dropout_rate=0.1,
        dense_units=64,
        learning_rate=0.001,
        batch_size=32,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5
)

# Add the results for Experiment 7 - BiLSTM to the results list
results_2.append(('E7 BiLSTM - afinn | vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=afinn_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    dense_units=1,
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=True, 
    use_kfold=True
)

# Add the results for Experiment 7 - MLP baseline to the results list
results_2.append(('E7 MLP - afinn | vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Write the results to the CSV file
with open(filename_2, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header_2)
    writer.writerows(results_2)

No sentence-transformers model found with name C:\Users\beren/.cache\torch\sentence_transformers\bert-base-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\beren/.cache\torch\sentence_transformers\bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

Majority baseline for vote_positive, vote_negative:
  Majority class: vote_positive
  Baseline accuracy: 53.33%

Majority baseline for vader_positive, vader_negative:
  Majority class: vader_positive
  Baseline accuracy: 71.11%

Majority baseline for textblob_positive, textblob_negative:
  Majority class: textblob_positive
  Baseline accuracy: 77.78%

Majority baseline for afinn_positive, afinn_negative:
  Majority class: afinn_positive
  Baseline accuracy: 75.56%

Epoch 1/10
2/2 [==============================] - 5s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 6/10
2/2 [==============

C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
2/2 [==============================] - 5s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 8/10
2/2 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 9/10
2/2 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 10/10
1/1 [==============================] - 1s 502ms/step


C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 4ms/step - loss: 1.0779 - accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 1.0371 - accuracy: 0.2500
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 0.9950 - accuracy: 0.2500
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 0.9536 - accuracy: 0.2500
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.9149 - accuracy: 0.2500
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.8783 - accuracy: 0.2500
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.8467 - accuracy: 0.2500
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 0.8184 - accuracy: 0.2500
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.7941 - accuracy: 0.2500
Epoch 10/10
1/1 [==============================] - 0s 63ms/step
Epoch 1/10


C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2/2 [==============================] - 0s 3ms/step - loss: 0.6396 - accuracy: 0.7778
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.6334 - accuracy: 0.7778
Epoch 3/10
2/2 [==============================] - 0s 5ms/step - loss: 0.6281 - accuracy: 0.7778
Epoch 4/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6237 - accuracy: 0.7778
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6195 - accuracy: 0.7778
Epoch 6/10
2/2 [==============================] - 0s 3ms/step - loss: 0.6158 - accuracy: 0.7778
Epoch 7/10
2/2 [==============================] - 0s 3ms/step - loss: 0.6126 - accuracy: 0.7778
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6096 - accuracy: 0.7778
Epoch 9/10
2/2 [==============================] - 0s 4ms/step - loss: 0.6074 - accuracy: 0.7778
Epoch 10/10
1/1 [==============================] - 0s 59ms/step
Average accuracy: 48.89%
Average precision: 25.19%
Average recall: 48.89%
Average F

C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
view = pd.read_csv(filename_2)
view

,Experiment,Accuracy,Precision,Recall,F1,Majority baseline,Parameters
0,E1 BiLSTM - vote | vote,0.533333,0.286420,0.533333,0.372161,0.533333,"LSTM_units = 64, \n spatial_dropout = 0.3, \n ..."
1,E1 MLP - vote | vote,0.533333,0.286420,0.533333,0.372161,0.533333,Standard
2,E2 BiLSTM - vader | vader,0.711111,0.508642,0.711111,0.592222,0.711111,"LSTM_units = 64, \n spatial_dropout = 0.2, \n ..."
3,E2 MLP - vader | vader,0.711111,0.508642,0.711111,0.592222,0.711111,Standard
4,E3 BiLSTM - vader | vote,0.533333,0.296296,0.533333,0.378095,0.533333,"LSTM_units = 64, \n spatial_dropout = 0.3, \n ..."
5,E3 MLP - vader | vote,0.533333,0.296296,0.533333,0.378095,0.533333,Standard
6,E4 BiLSTM - textblob | textblob,0.777778,0.604938,0.777778,0.680556,0.777778,"LSTM_units = 64, \n spatial_dropout = 0.2, \n ..."
7,E4 MLP - textblob | textblob,0.377778,0.276173,0.377778,0.282143,0.777778,Standard
8,E5 BiLSTM - textblob | vote,0.533333,0.286420,0.533333,0.372161,0.533333,"LSTM_units = 64, \n spatial_dropout = 0.3, \n ..."
9,E5 MLP - textblob | vote,0.511111,0.264198,0.511111,0.347497,0.533333,Standard


In [ ]:
token = pd.read_csv('removed_512.csv')
token

,original,>512 removed
0,"(100, 11)","(57, 12)"


In [7]:
rem = pd.read_csv('removed_neutral.csv')
rem

,original,neutral removed
0,"(57, 15)","(45, 15)"


In [ ]:
perc = pd.read_csv('percentages.csv')
perc

,column,positive,negative
0,vote,53.333333,46.666667
1,vader,71.111111,28.888889
2,textblob,77.777778,22.222222
3,afinn,75.555556,24.444444
